In [56]:
import mediapipe as mp
import cv2
import numpy as np
import os

In [57]:
# Inisialisasi MediaPipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True)

In [58]:
# Fungsi untuk menempatkan gambar hewan pada wajah
def overlay_animal(image, landmarks, animal_image, scale_factor=0.4, max_height=200):
    h, w, _ = image.shape
    ah, aw, _ = animal_image.shape

    # Landmark untuk mulut
    top_lip = landmarks[13]  # Landmark bibir atas
    bottom_lip = landmarks[14]  # Landmark bibir bawah

    # Koordinat mulut pengguna
    mouth_top = (int(top_lip.x * w), int(top_lip.y * h))
    mouth_bottom = (int(bottom_lip.x * w), int(bottom_lip.y * h))
    mouth_center = (mouth_top[0], (mouth_top[1] + mouth_bottom[1]) // 2)

    # Hitung jarak pembukaan mulut
    mouth_opening = abs(mouth_bottom[1] - mouth_top[1])

    # Pastikan pembukaan mulut cukup besar untuk mengubah ukuran gambar
    if mouth_opening < 5:  # Ambang batas pembukaan mulut yang terlalu kecil
        mouth_opening = 5  # Setel ke nilai minimum yang valid

    # Sesuaikan tinggi gambar katak dengan pembukaan mulut secara vertikal
    new_height = int(ah + mouth_opening * scale_factor)  # Tinggi bertambah seiring pembukaan mulut
    new_width = int(aw * scale_factor)  # Lebar tetap, hanya tinggi yang berubah

    # Batasi tinggi gambar katak agar tidak terlalu besar
    new_height = min(new_height, max_height)  # Batasi tinggi maksimal

    # Pastikan ukuran baru tidak kurang dari 1
    new_height = max(new_height, 1)
    new_width = max(new_width, 1)

    resized_frog = cv2.resize(animal_image, (new_width, new_height))  # Sesuaikan ukuran
    ah, aw, _ = resized_frog.shape

    # Tempelkan gambar katak di sekitar mulut
    y_offset = mouth_center[1] - ah // 2
    x_offset = mouth_center[0] - aw // 2

    # Validasi koordinat agar tidak keluar batas
    y1, y2 = max(0, y_offset), min(h, y_offset + ah)
    x1, x2 = max(0, x_offset), min(w, x_offset + aw)

    # Sesuaikan ukuran overlay agar sesuai dengan frame
    overlay_y1, overlay_y2 = max(0, -y_offset), ah - max(0, (y_offset + ah) - h)
    overlay_x1, overlay_x2 = max(0, -x_offset), aw - max(0, (x_offset + aw) - w)

    for c in range(3):  # Untuk setiap channel warna (RGB)
        alpha = resized_frog[overlay_y1:overlay_y2, overlay_x1:overlay_x2, 3] / 255.0
        image[y1:y2, x1:x2, c] = (
            alpha * resized_frog[overlay_y1:overlay_y2, overlay_x1:overlay_x2, c]
            + (1 - alpha) * image[y1:y2, x1:x2, c]
        )

    return image

In [59]:
# Path ke gambar hewan (katak)
animal_image_path = os.path.join(os.getcwd(), 'data', 'frog.png')

# Muat gambar hewan (katak dengan transparansi)
animal_image = cv2.imread(animal_image_path, cv2.IMREAD_UNCHANGED)

In [60]:
# Mulai kamera
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("Tidak dapat mengakses kamera.")
        break

    # Konversi gambar ke RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    # Jika wajah terdeteksi
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            landmarks = face_landmarks.landmark
            frame = overlay_animal(frame, landmarks, animal_image)

    # Tampilkan hasil
    cv2.imshow("Animal Filter", frame)

    # Tombol 'ESC' untuk keluar
    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()